In [2]:
import tensorflow as tf
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [3]:
# 층 : 유용한 연산자 집합

layer = tf.keras.layers.Dense(100)
layer = tf.keras.layers.Dense(10, input_shape = (None, 5))

In [4]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [6]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 1.2684220e-01, -3.9093733e-02, -6.0275763e-01, -5.3911948e-01,
          1.6613197e-01, -5.9996408e-01, -4.5684618e-01,  4.0619701e-01,
         -2.9163799e-01,  1.8462366e-01],
        [ 1.9773239e-01, -5.9227276e-01,  2.5135815e-02,  3.3232164e-01,
         -5.4760027e-01,  6.0555488e-01, -3.8353673e-01,  9.9241734e-05,
         -5.6716204e-03, -1.6920051e-01],
        [-5.6690037e-01, -6.0184097e-01, -8.3577275e-02,  1.7103016e-01,
         -2.0711416e-01,  4.7971231e-01,  1.9997954e-03,  2.8028882e-01,
         -6.2376225e-01,  4.0749866e-01],
        [ 4.6547407e-01, -4.1803578e-01, -2.7435619e-01, -4.3708828e-01,
          5.2985126e-01, -1.2161702e-01,  2.6694822e-01, -4.2851064e-01,
         -2.7404240e-01,  4.4889146e-01],
        [ 6.0676634e-02,  1.9377810e-01, -4.5542982e-01,  1.1409795e-01,
          1.9123548e-01, -4.9611092e-01,  3.6402458e-01,  7.7004671e-02,
          5.1428825e-01,  3.56432

In [7]:
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 1.2684220e-01, -3.9093733e-02, -6.0275763e-01, -5.3911948e-01,
          1.6613197e-01, -5.9996408e-01, -4.5684618e-01,  4.0619701e-01,
         -2.9163799e-01,  1.8462366e-01],
        [ 1.9773239e-01, -5.9227276e-01,  2.5135815e-02,  3.3232164e-01,
         -5.4760027e-01,  6.0555488e-01, -3.8353673e-01,  9.9241734e-05,
         -5.6716204e-03, -1.6920051e-01],
        [-5.6690037e-01, -6.0184097e-01, -8.3577275e-02,  1.7103016e-01,
         -2.0711416e-01,  4.7971231e-01,  1.9997954e-03,  2.8028882e-01,
         -6.2376225e-01,  4.0749866e-01],
        [ 4.6547407e-01, -4.1803578e-01, -2.7435619e-01, -4.3708828e-01,
          5.2985126e-01, -1.2161702e-01,  2.6694822e-01, -4.2851064e-01,
         -2.7404240e-01,  4.4889146e-01],
        [ 6.0676634e-02,  1.9377810e-01, -4.5542982e-01,  1.1409795e-01,
          1.9123548e-01, -4.9611092e-01,  3.6402458e-01,  7.7004671e-02,
          5.1428825e-01,  3.56432

### 사용자 정의 층 구현

1. __ init __ : 모든 입력 독립적 초기화를 수행할 수 있음 /  __ init __ 에 변수를 생성하는 경우 변수 생성에 필요한 크기가 명시적으로 지정되어야 함
2. build : 입력 텐서의 형상을 알고 나머지 초기화 작업을 수행할 수 있음 / build에 변수를 생성하는 경우 유리한 점은 층이 작동할 입력의 크기를 기준으로 나중에 변수를 만들 수 있음
3. call : 순방향 계산을 수행

In [10]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight('kernel', shape = [int(input_shape[-1]), self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [11]:
_ = layer(tf.zeros([10, 5]))

In [12]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer_1/kernel:0']


### 모델 : 층 구성

In [17]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [19]:
_ = block(tf.zeros([1,2,3,3]))

In [20]:
block.layers

In [21]:
len(block.variables)

18

In [22]:
block.summary()

Model: "resnet_identity_block_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_6 (Batch multiple                  4         
_________________________________________________________________
conv2d_7 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_7 (Batch multiple                  8         
_________________________________________________________________
conv2d_8 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_8 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
______________________________________________

In [23]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1,1), input_shape = (None, None, 3)),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(2, 1, padding = 'same'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(3, (1,1)),
                              tf.keras.layers.BatchNormalization()])

my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [24]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_9 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_10 (Batc (None, None, None, 2)     8         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_11 (Batc (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________